# Analyse_Models3

In [1]:
import pickle 
import numpy as np
import time 
import datetime as dt
from Multi_DeForest_Husky_C import *
from Prepare_Husky import *
from big_dix import *
from functools import reduce
import Add_HUSKY as H1
pd.set_option('display.max_rows',6000)
pd.set_option('display.max_columns',1000)

Este módulo usa pandas.Dataframe
E está feito para apenas prever variáveis categóricas 0/1
 
No DeForest, escolhemos as colunas a analisar por árvore
Ao contrário do Random_Forest, onde fazíamos random
big_dix tem funções para analisar os modelos
Prepare_Husky
Neste modelos vamos fazer uma experiência e não usar Odds
Em vex disso, usamos os xPTS para calcular o "power" de uma equipa


In [2]:
help(Analyse_Model)

Help on function Analyse_Model in module big_dix:

Analyse_Model(repos, w=4, odd=1)



In [2]:
rd0=open('Husky6_DFSA1_P2_S.pickle','rb')
F1=pickle.load(rd0)
Train1=pickle.load(rd0)
Test1=pickle.load(rd0)
T_df1=pickle.load(rd0)
rd0.close()

F2 = Filter_Forest(F1,Test1,T_df1)

print(F1.size(),'F1')
print(F2.size(),'F2')

C:\Users\joaom\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


Cagou no None
1331 F1
1100 F2


In [4]:
op0=open('Husky6_DFSA1_P2_S_F2.pickle','wb')
pickle.dump(F2,op0)
pickle.dump(Train1,op0)
pickle.dump(Test1,op0)
pickle.dump(T_df1,op0)
op0.close()

In [4]:
def classify_df2(DF,node,var=1):
    #var_list irá conter os índices onde a previsão da floresta foi 1
    #sevre para chegarmos as folhas
    #só serve para uma linha/amostra de dados serve 
    if isinstance(node,Leaf):
        prob=node.probability
        return [[prob[var],list(DF[:,-1])]] #-1 é o 'Index'                        
    t,f=partition(DF,node.question)
    T=classify_df(t,node.true_branch)
    F=classify_df(f,node.false_branch)
    return T+F #vamos fazendo return da soma das listas com os índices que vão ser previstos com 1


def prob_to_list2(w,ind): #serve para o reduce da info das probas (sem usar limite)
    print(w)
    for i in range(len(ind[1])):
        if ind[1][i]!=int(ind[1][i]):
            print(ind[1][i],'----',int(ind[1][i]))
            print('i',i)
        w[int(ind[1][i])]=ind[0]
    return w
    
def tree_proba2(df,forest,var=1):
    size=forest.size()
    for t in range(size):
        if t%10==0:
            print(t)
        node=forest.get_tree(t)    
        ko=classify_df2(df,node,var=var)
        ko2=np.array([list(reduce(prob_to_list2,ko,[0]*len(df)))])
        if t==0:
            tp2=np.array(ko2)
        else:
            tp2=np.concatenate((tp2,ko2))
                    
    return tp2.T

In [12]:
index=np.array([[inx] for inx in range(len(Test1))]) #criamos a coluna index
Test2 = np.append(Test1.values,index,axis=1)

In [13]:
classify_df2(Test2,F1.get_tree(1))

[[51.4, [18.0, 58.0, 82.0, 125.0]],
 [18.8, [61.0, 94.0]],
 [81.1, [12.0, 53.0]],
 [74.3, [5.0, 17.0, 36.0, 46.0, 55.0, 67.0, 69.0]],
 [58.1, [8.0, 27.0, 33.0, 47.0, 78.0, 111.0, 118.0]],
 [47.2, [59.0, 74.0, 120.0]],
 [57.5, [3.0, 10.0, 42.0, 50.0, 101.0]],
 [60.0, [89.0]],
 [38.1, [37.0]],
 [12.9, [30.0, 95.0, 107.0]],
 [53.8, [32.0]],
 [71.8, [9.0, 66.0, 73.0, 81.0, 90.0, 92.0, 98.0, 108.0, 114.0]],
 [31.6, [35.0, 49.0, 113.0]],
 [29.6, [11.0, 23.0]],
 [56.8, [1.0, 4.0, 13.0, 43.0, 77.0, 88.0, 97.0, 102.0, 105.0, 106.0, 115.0]],
 [26.7, [6.0, 29.0, 51.0, 56.0, 79.0, 93.0, 109.0, 126.0]],
 [36.3,
  [2.0,
   19.0,
   26.0,
   38.0,
   39.0,
   41.0,
   44.0,
   52.0,
   72.0,
   80.0,
   84.0,
   86.0,
   87.0,
   110.0,
   112.0,
   117.0,
   122.0,
   128.0]],
 [30.4,
  [7.0,
   15.0,
   20.0,
   24.0,
   28.0,
   45.0,
   48.0,
   60.0,
   65.0,
   68.0,
   71.0,
   83.0,
   85.0,
   91.0,
   96.0,
   103.0,
   119.0,
   124.0,
   127.0,
   132.0]],
 [17.3, [0.0, 22.0, 25.0, 31.0, 

In [19]:
print(F1.size(),'F1')
print(F2.size(),'F2')

681 F1
448 F2


In [24]:
Test11 = Test1[:int(len(Test1)/2)]
Test21 = Test1[int(len(Test1)/2):]

In [30]:
F3 = Optimize_Forest(F2,Test11,T_df1) 

Cagou no None


In [50]:
Test2 = Give_Test2('Bund',1)

In [59]:
Test_all = pd.concat([Test1,Test2],sort=False).sort_values(['Date','Time'],ascending=[True,True]).reset_index(drop=True)

In [18]:
Test1['ODDH_Aver.']=list(map(lambda x:float(x),list(Test1['ODDH_Aver.'])))

In [8]:
repos_Scott_P2_S = Big_dix(Test1,F2,T_df1)

Cagou no None
T50
T51
T52
T53
T54
T55
T56
T57
T58
T59
T60
T61
T62
T63
T64
T65
T66
T67
T68
T69
T70


KeyboardInterrupt: 

In [41]:
go_get(repos_Scott_P2_S,mode=2,lista=['Tree_Forest','TL56','FL30','Not_Predicted'])#.sort_values(['Tree Score %'],ascending=[False])

,Index,League,Date,Time,HT,AT,Tree Score %,Tree_var%56,ForestPredix_30%,ForestReal_30%,Won/NotWon,ODDH_Aver.,ODDD_Aver.,ODDA_Aver.
0,0,Scottish Premiership,2018-12-08,1500,Livingston,St Mirren,42.63,17.41,0,----(1),1,1.80,3.48,4.74
1,1,Scottish Premiership,2018-12-09,1330,Dundee,Rangers,37.37,11.16,0,----(0),0,9.42,5.32,1.31
2,2,Scottish Premiership,2018-12-11,1945,Aberdeen,Livingston,42.09,15.62,0,----(1),1,1.88,3.18,4.81
3,3,Scottish Premiership,2018-12-14,1945,Livingston,Hearts,43.62,22.77,0,----(1),1,3.17,3.16,2.37
4,4,Scottish Premiership,2018-12-15,1500,Kilmarnock,Dundee,41.04,15.62,0,----(1),1,1.60,3.84,5.85
5,5,Scottish Premiership,2018-12-15,1500,St. Johnstone,Motherwell,49.01,29.69,0,----(0),0,2.01,3.42,3.78
6,6,Scottish Premiership,2018-12-15,1500,St Mirren,Aberdeen,36.62,16.52,0,----(0),0,5.21,3.62,1.71
7,7,Scottish Premiership,2018-12-16,1230,Hibernian,Celtic,28.10,5.13,0,----(1),1,6.84,4.38,1.47
11,11,Scottish Premiership,2018-12-19,1945,Hibernian,Rangers,40.83,23.21,0,----(0),0,4.20,3.75,1.82
14,14,Scottish Premiership,2018-12-22,1500,Hamilton Academical,Kilmarnock,25.75,6.92,0,----(0),0,4.80,3.73,1.74


In [36]:
dfgg = dfgg.reset_index(drop=True)
dfgg.loc[np.array(np.where(np.logical_or(np.logical_and(dfgg['HT']=='Celtic',dfgg['AT']=='Rangers'),
                                    np.logical_and(dfgg['HT']=='Rangers',dfgg['AT']=='Celtic')))[0])]

,Index,League,Date,Time,HT,AT,Tree Score %,Tree_var%50,ForestPredix_30%,ForestReal_30%,Won/NotWon,ODDH_Aver.,ODDD_Aver.,ODDA_Aver.
26,125,Scottish Premiership,2019-05-12,1200,Rangers,Celtic,50.71,45.76,1,YES,1,2.52,3.39,2.77
48,90,Scottish Premiership,2019-03-31,1200,Celtic,Rangers,45.49,29.24,0,----(1),1,1.83,3.55,4.45
75,24,Scottish Premiership,2018-12-29,1230,Rangers,Celtic,40.62,29.46,0,----(1),1,3.00,3.21,2.46


In [37]:
dfgg

,Index,League,Date,Time,HT,AT,Tree Score %,Tree_var%50,ForestPredix_30%,ForestReal_30%,Won/NotWon,ODDH_Aver.,ODDD_Aver.,ODDA_Aver.
0,35,Scottish Premiership,2019-01-23,1945,St. Johnstone,Livingston,64.28,69.64,1,YES,1,2.17,2.94,3.96
1,74,Scottish Premiership,2019-03-09,1500,Celtic,Aberdeen,62.42,68.75,1,NO,0,1.38,4.66,8.76
2,58,Scottish Premiership,2019-02-16,1500,Rangers,St. Johnstone,61.63,72.99,1,NO,0,1.30,5.49,9.62
3,9,Scottish Premiership,2018-12-18,1945,Aberdeen,Dundee,60.21,63.62,1,YES,1,1.53,3.98,6.65
4,30,Scottish Premiership,2019-01-23,1945,Celtic,St Mirren,59.92,59.15,1,YES,1,1.10,9.32,26.86
5,12,Scottish Premiership,2018-12-22,1500,Aberdeen,Hearts,59.65,69.87,1,YES,1,1.95,3.44,3.99
6,13,Scottish Premiership,2018-12-22,1500,Celtic,Dundee,59.61,66.74,1,YES,1,1.10,9.61,22.67
7,53,Scottish Premiership,2019-02-06,1945,Hearts,Livingston,59.19,67.41,1,NO,0,1.74,3.53,5.00
8,82,Scottish Premiership,2019-03-16,1500,Rangers,Kilmarnock,58.82,61.38,1,NO,0,1.41,4.50,8.08
9,37,Scottish Premiership,2019-01-26,1500,Celtic,Hamilton Academical,58.29,60.04,1,YES,1,1.08,10.69,30.68


# Scott

In [23]:
#df_scott_p2_s=Analyse_Model(repos_Scott_P2_S,odd=1)
df_scott_p2_s.head(20)
#df_scott_p2_s.sort_values(['Week 4'],ascending=[False])
#Aux_All1(df_scott_p2_s,All=0.75).head(10)
#df_Scott = go_get(repos_Scott_P2_S,mode=2,lista=['Tree_Forest','TL60','FL26','Predicted'])

,Method,Week 4,Week 8,Week 12,Week 16,Week 20,Week 24,Total,Method2,Size,Max0,Max1,Aver_0,Aver_1,STD_0,STD_1,ODD_12,ODD_15,ODD_2
0,TL50_FL25,0.88,1.13,1.04,1.19,1.24,1.44,2.21,TL50_FL25,76,5,7,1.6,2.4,16.0,19.0,88.16,67.11,39.47
1,TL51_FL23,0.88,1.13,1.04,1.19,1.24,1.44,2.21,TL51_FL23,76,5,7,1.6,2.4,16.0,19.0,88.16,67.11,39.47
2,TL60_FL26,1.07,1.05,1.40,0.93,1.12,1.30,2.13,TL60_FL26,36,3,7,1.4,3.2,4.0,10.0,75.00,44.44,27.78
3,TL56_FL31,1.19,1.05,1.22,0.93,1.12,1.33,2.12,TL56_FL31,36,3,7,1.3,3.9,5.0,11.0,75.00,38.89,22.22
4,TL53_FL25,0.83,1.25,1.16,0.93,1.43,1.30,2.09,TL53_FL25,65,4,6,1.5,2.4,12.0,17.0,86.15,61.54,38.46
5,TL59_FL28,1.19,1.05,1.22,0.93,1.12,1.30,2.08,TL59_FL28,35,3,7,1.3,3.7,5.0,10.0,74.29,40.00,22.86
6,TL54_FL24,0.69,1.25,1.16,1.11,1.43,1.30,2.08,TL54_FL24,65,4,6,1.5,2.4,12.0,16.0,86.15,61.54,38.46
7,TL50_FL26,0.82,1.25,1.04,1.19,1.24,1.30,2.06,TL50_FL26,73,5,7,1.6,2.4,15.0,18.0,87.67,65.75,38.36
8,TL56_FL30,1.07,1.05,1.22,0.93,1.12,1.43,2.05,TL56_FL30,38,3,6,1.2,3.5,5.0,11.0,76.32,42.11,23.68
9,TL57_FL29,1.07,1.05,1.22,0.93,1.12,1.43,2.05,TL57_FL29,38,3,6,1.2,3.5,5.0,11.0,76.32,42.11,23.68


# Eredivisie

In [36]:
#df_ered_p2_s=Analyse_Model(repos_Ered_P2_S,odd=1)
#df_ered_p2_s.head(10)
Aux_All1(df_ered_p2_s,All=0.7).head(5)
#testing=df_ered_p2_s.sort_values(['Week 4'],ascending=[False]).reset_index(drop=True)
#testing.loc[testing['Method']=='TL50_FL25']
#df_Ered = go_get(repas_Ered_P2_S,mode=2,lista=['Tree_Forest','TL56','FL26','Predicted'])

,Method,Week 4,Week 8,Week 12,Week 16,Week 20,Total,Method2,Size,Max0,Max1,Aver_0,Aver_1,STD_0,STD_1,ODD_12,ODD_15,ODD_2
0,TL56_FL26,1.91,0.88,1.30,1.24,1.15,3.12,TL56_FL26,99,6,11,1.5,2.8,18.0,32.0,88.89,63.64,40.40
1,TL64_FL16,1.91,0.88,1.45,1.24,1.04,3.11,TL64_FL16,101,5,12,1.6,2.8,18.0,34.0,89.11,64.36,41.58
2,TL61_FL19,1.91,0.79,1.45,1.24,1.15,3.11,TL61_FL19,101,5,12,1.6,2.8,18.0,34.0,89.11,64.36,41.58
3,TL56_FL27,1.64,0.87,1.45,1.24,1.15,2.98,TL56_FL27,96,6,11,1.6,3.0,17.0,34.0,88.54,62.50,38.54
4,TL53_FL31,1.64,0.79,1.45,1.38,1.15,2.97,TL53_FL31,98,6,12,1.6,3.0,18.0,36.0,88.78,63.27,39.80


# Championship

In [417]:
#df_champ_p2_s=Analyse_Model(repos_Champ_P2_S)
#df_champ_p2_s.sort_values(['Week 8'],ascending=[False])
#Aux_All1(g2df_ll_p2_s,All=0.9).(20)

# LL

In [359]:
#g2df_ll_p2_s=Analyse_Model(G2repos_LL_P2_S)
Aux_All1(g2df_ll_p2_s,All=0.9).head(20)

,Method,Week 4,Week 8,Week 12,Week 16,Week 20,Week 24,Total,Method2,Size,Max0,Max1,Aver_0,Aver_1,STD_0,STD_1,ODD_12,ODD_15,ODD_2
0,TL92_FL18,1.08,1.15,0.91,1.01,1.22,1.0,1.40,TL92_FL18,17,1,7,0.7,5.0,1.0,6.0,64.71,29.41,5.88
1,TL95_FL14,1.08,1.15,0.91,1.01,1.22,1.0,1.40,TL95_FL14,17,1,7,0.7,5.0,1.0,6.0,64.71,29.41,5.88
2,TL93_FL16,1.08,1.15,0.91,1.01,1.22,1.0,1.40,TL93_FL16,17,1,7,0.7,5.0,1.0,6.0,64.71,29.41,5.88
3,TL92_FL17,1.08,1.15,0.91,1.01,1.22,1.0,1.40,TL92_FL17,17,1,7,0.7,5.0,1.0,6.0,64.71,29.41,5.88
4,TL95_FL13,1.08,1.15,0.97,0.91,1.22,1.0,1.34,TL95_FL13,19,1,8,0.8,4.0,2.0,10.0,68.42,31.58,5.26
5,TL94_FL14,1.08,1.15,0.97,0.91,1.22,1.0,1.34,TL94_FL14,19,1,8,0.8,4.0,2.0,10.0,68.42,31.58,5.26
6,TL96_FL12,1.08,1.15,0.97,0.91,1.22,1.0,1.34,TL96_FL12,19,1,8,0.8,4.0,2.0,10.0,68.42,31.58,5.26
7,TL94_FL17,1.08,1.15,0.91,1.01,1.15,1.0,1.33,TL94_FL17,16,1,7,0.7,4.7,1.0,5.0,62.50,25.00,6.25
8,TL95_FL15,1.08,1.15,0.91,1.01,1.15,1.0,1.33,TL95_FL15,16,1,7,0.7,4.7,1.0,5.0,62.50,25.00,6.25
9,TL94_FL16,1.08,1.15,0.91,1.01,1.15,1.0,1.33,TL94_FL16,16,1,7,0.7,4.7,1.0,5.0,62.50,25.00,6.25


In [392]:
#df_ll_p2_s = Analyse_Model(repos_LL_P2_S)
#Aux_All1(df_ll_p2_s,All=0.75).head(10)
#df_ll_p2_s.loc[(df_ll_p2_s['Max0']<=2)].sort_values('Total',ascending=False).head(20)
#df_LL = go_get(repos_LL_P2_S,mode=2,lista=['Tree_Forest','TL95','FL11','Predicted'])
df_ll_p2_s.sort_values('Week 4',ascending=False).head(10)

,Method,Week 4,Week 8,Week 12,Week 16,Week 20,Week 24,Total,Method2,Size,Max0,Max1,Aver_0,Aver_1,STD_0,STD_1,ODD_12,ODD_15,ODD_2
66,TL55_FL27,2.88,0.73,0.48,1.50,1.06,1.10,1.75,TL55_FL27,147,5,7,1.8,2.2,33.0,46.0,95.24,81.63,49.66
13,TL87_FL8,2.87,0.60,0.73,1.80,0.90,1.36,2.77,TL87_FL8,115,4,7,1.5,2.2,22.0,41.0,93.91,76.52,38.26
26,TL85_FL9,2.87,0.54,0.73,1.80,0.90,1.36,2.49,TL85_FL9,116,4,7,1.5,2.2,23.0,41.0,93.97,76.72,38.79
24,TL86_FL9,2.87,0.67,0.73,2.00,0.90,1.03,2.58,TL86_FL9,112,4,7,1.5,2.4,22.0,39.0,93.75,75.89,36.61
17,TL88_FL8,2.87,0.60,0.73,1.78,0.90,1.36,2.73,TL88_FL8,113,4,7,1.5,2.3,22.0,40.0,93.81,76.11,37.17
14,TL86_FL8,2.87,0.60,0.73,1.80,0.90,1.36,2.77,TL86_FL8,115,4,7,1.5,2.2,22.0,41.0,93.91,76.52,38.26
42,TL85_FL8,2.82,0.61,0.59,1.62,0.90,1.36,2.02,TL85_FL8,122,4,7,1.6,2.3,27.0,42.0,94.26,77.87,40.98
49,TL84_FL9,2.82,0.54,0.66,1.62,0.90,1.36,1.98,TL84_FL9,120,4,7,1.6,2.2,26.0,42.0,94.17,77.50,40.00
35,TL82_FL9,2.81,0.54,0.59,2.23,0.82,1.36,2.22,TL82_FL9,127,5,7,1.6,2.2,29.0,43.0,94.49,78.74,41.73
2830,TL61_FL23,2.81,0.68,0.65,1.23,0.66,0.93,0.94,TL61_FL23,127,4,7,1.8,2.3,28.0,46.0,94.49,78.74,42.52


In [368]:
#df_ll2_p2_s=Analyse_Model(repos_LL2_P2_S)
#df_ll2_p2_s.loc[df_ll2_p2_s['Max0']<20].head(10)
#Aux_All1(df_ll2_p2_s,All=0.8).head(10)
df_ll2_p2_s.head(10)

,Method,Week 4,Week 8,Week 12,Week 16,Week 20,Week 24,Total,Method2,Size,Max0,Max1,Aver_0,Aver_1,STD_0,STD_1,ODD_12,ODD_15,ODD_2
0,TL94_FL7,1.84,0.88,0.71,1.18,1.77,0.90,2.15,TL94_FL7,73,4,7,1.4,2.6,12.0,25.0,90.41,64.38,27.40
1,TL85_FL7,2.05,0.55,0.59,1.42,1.37,1.36,1.76,TL85_FL7,134,5,6,1.7,2.1,31.0,43.0,94.78,79.85,45.52
2,TL95_FL7,1.97,0.82,0.76,1.01,1.56,0.90,1.76,TL95_FL7,60,4,5,1.4,2.3,10.0,20.0,88.33,61.67,30.00
3,TL85_FL9,3.12,0.60,0.70,1.38,0.87,1.03,1.62,TL85_FL9,111,4,7,1.5,2.3,22.0,37.0,93.69,75.68,36.04
4,TL86_FL9,3.25,0.60,0.70,1.28,0.91,1.03,1.62,TL86_FL9,106,4,7,1.5,2.3,21.0,35.0,93.40,74.53,35.85
5,TL73_FL13,2.27,0.87,0.65,1.31,0.90,1.03,1.57,TL73_FL13,132,4,7,1.6,2.2,30.0,43.0,94.70,79.55,43.94
6,TL88_FL7,2.57,0.54,0.63,1.15,1.52,1.03,1.57,TL88_FL7,115,5,7,1.6,2.3,27.0,36.0,93.91,76.52,39.13
7,TL54_FL24,2.59,0.54,0.43,1.21,1.85,1.10,1.49,TL54_FL24,165,5,6,1.8,2.1,41.0,42.0,95.76,83.64,52.73
8,TL56_FL23,2.59,0.54,0.43,1.21,1.85,1.10,1.49,TL56_FL23,165,5,6,1.8,2.1,41.0,42.0,95.76,83.64,52.73
9,TL89_FL7,2.08,0.54,0.70,1.28,1.38,1.03,1.42,TL89_FL7,106,5,7,1.6,2.2,23.0,33.0,93.40,74.53,38.68


# PL

In [5]:
#df3_pl_p2_s = Analyse_Model(repos3_PL_P2_S).head(200)
df3_pl_p2_s

,Method,Week 4,Week 8,Week 12,Week 16,Week 20,Total,Method2,Size,Max0,Max1,Aver_0,Aver_1,STD_0,STD_1,ODD_12,ODD_15,ODD_2
0,TL96_FL14,1.00,1.00,1.00,1.01,1.02,1.04,TL96_FL14,2,0,2,0.0,2.0,0.0,0.0,50.00,0.00,0.0
1,TL96_FL15,1.00,1.00,1.00,1.01,1.02,1.04,TL96_FL15,2,0,2,0.0,2.0,0.0,0.0,50.00,0.00,0.0
2,TL52_FL58,1.15,0.81,0.98,0.94,1.21,1.03,TL52_FL58,26,2,6,1.0,3.3,2.0,8.0,76.92,19.23,0.0
3,TL98_FL12,1.00,1.00,1.00,1.00,1.02,1.02,TL98_FL12,1,0,1,0.0,1.0,0.0,0.0,100.00,0.00,0.0
4,TL98_FL11,1.00,1.00,1.00,1.00,1.02,1.02,TL98_FL11,1,0,1,0.0,1.0,0.0,0.0,100.00,0.00,0.0
5,TL100_FL9,1.00,1.00,1.00,1.00,1.02,1.02,TL100_FL9,1,0,1,0.0,1.0,0.0,0.0,100.00,0.00,0.0
6,TL100_FL8,1.00,1.00,1.00,1.00,1.02,1.02,TL100_FL8,1,0,1,0.0,1.0,0.0,0.0,100.00,0.00,0.0
7,TL97_FL13,1.00,1.00,1.00,1.00,1.02,1.02,TL97_FL13,1,0,1,0.0,1.0,0.0,0.0,100.00,0.00,0.0
8,TL97_FL14,1.00,1.00,1.00,1.00,1.02,1.02,TL97_FL14,1,0,1,0.0,1.0,0.0,0.0,100.00,0.00,0.0
9,TL99_FL9,1.00,1.00,1.00,1.00,1.02,1.02,TL99_FL9,1,0,1,0.0,1.0,0.0,0.0,100.00,0.00,0.0


In [389]:
#df_pl_p2_s = Analyse_Model(repos_PL_P2_S)
#Aux_All1(df_pl_p2_s,All=0.1).head(10)
#df_PL = go_get(repos_PL_P2_S,mode=2,lista=['Tree_Forest','TL76','FL41','Predicted'])
#df_pl_p2_s_t2 = Analyse_Model(repos_PL_P2_S_T2)
#df_pl_p2_s_t2.loc[df_pl_p2_s_t2['Method']=='TL76_FL41']
#go_get(repos_PL_P2_S_T2,mode=2,lista=['Tree_Forest','TL76','FL41','Predicted'])
df_pl_p2_s.sort_values('Week 4',ascending=False).head(10)

,Method,Week 4,Week 8,Week 12,Week 16,Week 20,Total,Method2,Size,Max0,Max1,Aver_0,Aver_1,STD_0,STD_1,ODD_12,ODD_15,ODD_2
3042,TL96_FL13,1.46,0.75,1.02,0.86,0.98,0.93,TL96_FL13,30,2,6,1.4,2.9,5.0,13.0,90.00,53.33,10.00
3311,TL85_FL25,1.42,0.73,0.77,0.88,1.04,0.73,TL85_FL25,36,3,10,2.0,4.0,4.0,18.0,83.33,30.56,5.56
3326,TL86_FL24,1.42,0.65,0.83,0.88,1.04,0.71,TL86_FL24,38,3,8,1.9,3.6,5.0,17.0,84.21,31.58,5.26
3249,TL84_FL26,1.42,0.76,0.74,0.88,1.10,0.79,TL84_FL26,41,4,8,2.0,3.9,6.0,17.0,85.37,36.59,7.32
3238,TL85_FL24,1.42,0.69,0.83,0.88,1.12,0.80,TL85_FL24,41,3,8,2.0,3.9,4.0,17.0,85.37,36.59,9.76
3305,TL83_FL27,1.42,0.76,0.69,0.88,1.10,0.73,TL83_FL27,40,4,7,2.0,3.7,6.0,16.0,85.00,35.00,7.50
3234,TL92_FL19,1.41,0.84,0.75,0.88,1.04,0.81,TL92_FL19,31,4,8,1.7,3.5,6.0,15.0,80.65,35.48,6.45
3244,TL91_FL19,1.41,0.90,0.69,0.88,1.04,0.80,TL91_FL19,34,4,9,1.8,3.8,5.0,17.0,82.35,35.29,5.88
3309,TL100_FL7,1.40,0.77,0.69,0.94,1.04,0.73,TL100_FL7,23,3,5,1.5,2.3,6.0,7.0,91.30,52.17,17.39
3304,TL99_FL7,1.40,0.77,0.69,0.94,1.04,0.73,TL99_FL7,23,3,5,1.5,2.3,6.0,7.0,91.30,52.17,17.39


# Bund

In [61]:
df_bund_p2_tall = Analyse_Model(repos_Bund_P2_TAll)

TL50_FL1 weekly_profit
TL60_FL1 weekly_profit
TL70_FL1 weekly_profit
TL80_FL1 weekly_profit
TL90_FL1 weekly_profit
TL100_FL1 weekly_profit
TL50_FL1 weekly_profit
TL60_FL1 weekly_profit
TL70_FL1 weekly_profit
TL80_FL1 weekly_profit
TL90_FL1 weekly_profit
TL100_FL1 weekly_profit
TL50_FL1 weekly_profit
TL60_FL1 weekly_profit
TL70_FL1 weekly_profit
TL80_FL1 weekly_profit
TL90_FL1 weekly_profit
TL100_FL1 weekly_profit
TL50_FL1 weekly_profit
TL60_FL1 weekly_profit
TL70_FL1 weekly_profit
TL80_FL1 weekly_profit
TL90_FL1 weekly_profit
TL100_FL1 weekly_profit
TL50_FL1 weekly_profit
TL60_FL1 weekly_profit
TL70_FL1 weekly_profit
TL80_FL1 weekly_profit
TL90_FL1 weekly_profit
TL100_FL1 weekly_profit
TL50_FL1 weekly_profit
TL60_FL1 weekly_profit
TL70_FL1 weekly_profit
TL80_FL1 weekly_profit
TL90_FL1 weekly_profit
TL100_FL1 weekly_profit
TL50_FL1 weekly_profit
TL60_FL1 weekly_profit
TL70_FL1 weekly_profit
TL80_FL1 weekly_profit
TL90_FL1 weekly_profit
TL100_FL1 weekly_profit
TL50_FL1 weekly_profit
TL60

In [62]:
#df_bund_p2_t2 = Analyse_Model(repos_Bund_P2_T2)

In [63]:
#df_bund_p2 = Analyse_Model(repos_Bund_P2)

In [333]:
df_bund_p1 = Analyse_Model(repos_Bund_P1)

TL50_FL1 weekly_profit
TL60_FL1 weekly_profit
TL70_FL1 weekly_profit
TL80_FL1 weekly_profit
TL90_FL1 weekly_profit
TL100_FL1 weekly_profit
TL50_FL1 weekly_profit
TL60_FL1 weekly_profit
TL70_FL1 weekly_profit
TL80_FL1 weekly_profit
TL90_FL1 weekly_profit
TL100_FL1 weekly_profit
TL50_FL1 weekly_profit
TL60_FL1 weekly_profit
TL70_FL1 weekly_profit
TL80_FL1 weekly_profit
TL90_FL1 weekly_profit
TL100_FL1 weekly_profit
TL50_FL1 weekly_profit
TL60_FL1 weekly_profit
TL70_FL1 weekly_profit
TL80_FL1 weekly_profit
TL90_FL1 weekly_profit
TL100_FL1 weekly_profit
TL50_FL1 weekly_profit
TL60_FL1 weekly_profit
TL70_FL1 weekly_profit
TL80_FL1 weekly_profit
TL90_FL1 weekly_profit
TL100_FL1 weekly_profit
TL50_FL1 weekly_profit
TL60_FL1 weekly_profit
TL70_FL1 weekly_profit
TL80_FL1 weekly_profit
TL90_FL1 weekly_profit
TL100_FL1 weekly_profit
TL50_FL1 weekly_profit
TL60_FL1 weekly_profit
TL70_FL1 weekly_profit
TL80_FL1 weekly_profit
TL90_FL1 weekly_profit
TL100_FL1 weekly_profit
TL50_FL1 weekly_profit
TL60

In [42]:
#df_bund_p1_t2 = Analyse_Model(repos_Bund_P1_T2)

In [41]:
#df_bund_p1_o = Analyse_Model(repos_Bund_P1_O)

In [40]:
#df_bund_p1_tall = Analyse_Model(repos_Bund_P1_Tall)

In [17]:
def Aux_All1(AM_df,count=0,columns=None):
    if count==0:
        C=list(AM_df.columns)
        C = [x for x in C if 'Week' in x]
        return AM_df.loc[Aux_All1(AM_df,count=1,columns=C)].sort_values(['Total'],ascending=[False]).reset_index(drop=True)
    else:
        if len(columns)==1:
            return AM_df[columns[0]]>=1
        elif len(columns)==2:
            return np.logical_and(AM_df[columns[0]]>=1,AM_df[columns[1]]>=1)
        else:
            p1 = columns[int(len(columns)/2):]
            p2 = columns[:int(len(columns)/2)]
            return np.logical_and(Aux_All1(AM_df,count=1,columns=p1),Aux_All1(AM_df,count=1,columns=p2))

In [64]:
df_bund_p2_tall.head(20)

,Method,Week 4,Week 8,Week 12,Week 16,Week 20,Week 24,Week 28,Week 32,Week 36,Total,Method2,Size,Max0,Max1,Aver_0,Aver_1,STD_0,STD_1,ODD_12,ODD_15,ODD_2
0,TL58_FL37,1.18,1.01,0.95,1.13,1.25,1.0,1.0,0.62,1.87,1.86,TL58_FL37,64,3,8,1.6,3.3,12.0,25.0,84.38,56.25,29.69
1,TL63_FL31,1.04,0.99,1.06,1.25,1.13,1.0,1.0,0.62,1.87,1.79,TL63_FL31,61,3,13,1.9,4.2,9.0,31.0,83.61,52.46,27.87
2,TL69_FL22,1.29,0.92,1.06,1.13,1.13,1.0,1.0,0.56,1.87,1.67,TL69_FL22,67,4,8,1.8,3.8,12.0,21.0,85.07,55.22,28.36
3,TL62_FL31,1.04,1.01,1.06,1.13,1.13,1.0,1.0,0.62,1.87,1.64,TL62_FL31,64,3,8,1.8,3.6,11.0,23.0,84.38,54.69,28.12
4,TL65_FL27,1.04,0.99,1.06,1.13,1.13,1.0,1.0,0.62,1.87,1.61,TL65_FL27,62,3,9,1.8,3.8,10.0,26.0,83.87,53.23,27.42
5,TL61_FL31,1.22,1.01,0.95,1.13,1.13,1.0,1.0,0.62,1.69,1.56,TL61_FL31,67,3,8,1.8,3.4,11.0,22.0,85.07,55.22,29.85
6,TL61_FL32,1.22,0.89,0.95,1.13,1.13,1.0,1.0,0.62,1.87,1.53,TL61_FL32,65,3,8,1.7,3.3,12.0,21.0,84.62,55.38,29.23
7,TL67_FL24,1.10,0.89,1.06,1.13,1.13,1.0,1.0,0.62,1.87,1.53,TL67_FL24,64,3,8,1.8,3.6,11.0,21.0,84.38,54.69,26.56
8,TL64_FL25,1.29,1.48,0.95,1.02,1.01,1.0,1.0,0.41,1.97,1.50,TL64_FL25,77,6,8,1.9,3.2,17.0,26.0,87.01,58.44,33.77
9,TL67_FL23,1.29,0.92,1.06,1.13,1.13,1.0,1.0,0.56,1.69,1.50,TL67_FL23,68,4,8,1.9,3.8,11.0,21.0,85.29,54.41,27.94


In [67]:
Aux_All1(df_bund_p2_tall).head(10)

,Method,Week 4,Week 8,Week 12,Week 16,Week 20,Week 24,Week 28,Week 32,Week 36,Total,Method2,Size,Max0,Max1,Aver_0,Aver_1,STD_0,STD_1,ODD_12,ODD_15,ODD_2
0,TL85_FL43,1.0,1.01,1.0,1.02,1.0,1.0,1.0,1.0,1.04,1.08,TL85_FL43,3,0,3,0.0,3.0,0.0,0.0,66.67,0.0,0.0
1,TL84_FL45,1.0,1.00,1.0,1.02,1.0,1.0,1.0,1.0,1.04,1.06,TL84_FL45,2,0,2,0.0,2.0,0.0,0.0,100.00,0.0,0.0
2,TL77_FL48,1.0,1.00,1.0,1.02,1.0,1.0,1.0,1.0,1.04,1.06,TL77_FL48,2,0,2,0.0,2.0,0.0,0.0,100.00,0.0,0.0
3,TL78_FL50,1.0,1.00,1.0,1.02,1.0,1.0,1.0,1.0,1.04,1.06,TL78_FL50,2,0,2,0.0,2.0,0.0,0.0,100.00,0.0,0.0
4,TL81_FL48,1.0,1.00,1.0,1.02,1.0,1.0,1.0,1.0,1.04,1.06,TL81_FL48,2,0,2,0.0,2.0,0.0,0.0,100.00,0.0,0.0
5,TL81_FL47,1.0,1.00,1.0,1.02,1.0,1.0,1.0,1.0,1.04,1.06,TL81_FL47,2,0,2,0.0,2.0,0.0,0.0,100.00,0.0,0.0
6,TL78_FL49,1.0,1.00,1.0,1.02,1.0,1.0,1.0,1.0,1.04,1.06,TL78_FL49,2,0,2,0.0,2.0,0.0,0.0,100.00,0.0,0.0
7,TL79_FL49,1.0,1.00,1.0,1.02,1.0,1.0,1.0,1.0,1.04,1.06,TL79_FL49,2,0,2,0.0,2.0,0.0,0.0,100.00,0.0,0.0
8,TL79_FL48,1.0,1.00,1.0,1.02,1.0,1.0,1.0,1.0,1.04,1.06,TL79_FL48,2,0,2,0.0,2.0,0.0,0.0,100.00,0.0,0.0
9,TL76_FL51,1.0,1.00,1.0,1.02,1.0,1.0,1.0,1.0,1.04,1.06,TL76_FL51,2,0,2,0.0,2.0,0.0,0.0,100.00,0.0,0.0


In [286]:
df_Bund = go_get(repos_Bund_P2,mode=2,lista=['Tree_Forest','TL59','FL46','Predicted'])

In [48]:
Aux_All1(df_bund_p2).head(20)

,Method,Week 4,Week 8,Week 12,Week 16,Week 20,Week 24,Week 28,Week 32,Total,Method2,Size,Max0,Max1,Aver_0,Aver_1,STD_0,STD_1,ODD_12,ODD_15,ODD_2
0,TL59_FL46,1.04,1.07,1.06,1.11,1.16,1.0,1.0,1.05,1.61,TL59_FL46,22,1,16,1.0,4.8,0.0,22.0,72.73,36.36,18.18
1,TL57_FL50,1.04,1.07,1.06,1.11,1.16,1.0,1.0,1.05,1.61,TL57_FL50,22,1,16,1.0,4.8,0.0,22.0,72.73,36.36,18.18
2,TL54_FL54,1.04,1.07,1.03,1.11,1.16,1.0,1.0,1.05,1.56,TL54_FL54,21,1,15,1.0,4.5,0.0,21.0,71.43,38.10,19.05
3,TL57_FL49,1.04,1.07,1.06,1.05,1.16,1.0,1.0,1.05,1.51,TL57_FL49,24,1,10,1.0,4.0,0.0,18.0,75.00,37.50,16.67
4,TL56_FL50,1.04,1.07,1.06,1.05,1.16,1.0,1.0,1.05,1.51,TL56_FL50,24,1,10,1.0,4.0,0.0,18.0,75.00,37.50,16.67
5,TL54_FL53,1.04,1.10,1.06,1.00,1.16,1.0,1.0,1.05,1.49,TL54_FL53,24,1,11,1.0,4.0,0.0,18.0,75.00,37.50,16.67
6,TL55_FL52,1.04,1.10,1.06,1.00,1.16,1.0,1.0,1.05,1.49,TL55_FL52,24,1,11,1.0,4.0,0.0,18.0,75.00,37.50,16.67
7,TL56_FL52,1.04,1.03,1.06,1.06,1.16,1.0,1.0,1.05,1.46,TL56_FL52,19,1,13,1.0,4.0,0.0,18.0,73.68,42.11,21.05
8,TL53_FL55,1.04,1.10,1.03,1.00,1.16,1.0,1.0,1.05,1.45,TL53_FL55,23,1,10,1.0,3.8,0.0,17.0,73.91,39.13,17.39
9,TL56_FL51,1.04,1.07,1.06,1.00,1.16,1.0,1.0,1.05,1.44,TL56_FL51,23,1,10,1.0,3.8,0.0,17.0,73.91,39.13,17.39


In [22]:
Aux_All1(df_bund_p1_tall).head(7)

,Method,Week 4,Week 8,Week 12,Week 16,Week 20,Week 24,Week 28,Week 32,Week 36,Total,Method2,Size,Max0,Max1,Aver_0,Aver_1,STD_0,STD_1,ODD_12,ODD_15,ODD_2
0,TL56_FL70,1.0,1.03,1.06,1.05,1.01,1.0,1.0,1.02,1.18,1.39,TL56_FL70,14,0,14,0.0,14.0,0.0,0.0,50.00,7.14,0.0
1,TL56_FL72,1.0,1.03,1.06,1.05,1.01,1.0,1.0,1.02,1.18,1.39,TL56_FL72,14,0,14,0.0,14.0,0.0,0.0,50.00,7.14,0.0
2,TL57_FL69,1.0,1.03,1.06,1.05,1.01,1.0,1.0,1.02,1.18,1.39,TL57_FL69,14,0,14,0.0,14.0,0.0,0.0,50.00,7.14,0.0
3,TL56_FL71,1.0,1.03,1.06,1.05,1.01,1.0,1.0,1.02,1.18,1.39,TL56_FL71,14,0,14,0.0,14.0,0.0,0.0,50.00,7.14,0.0
4,TL55_FL76,1.0,1.03,1.06,1.05,1.01,1.0,1.0,1.02,1.15,1.36,TL55_FL76,13,0,13,0.0,13.0,0.0,0.0,46.15,7.69,0.0
5,TL52_FL79,1.0,1.03,1.06,1.05,1.01,1.0,1.0,1.02,1.15,1.36,TL52_FL79,13,0,13,0.0,13.0,0.0,0.0,46.15,7.69,0.0
6,TL50_FL81,1.0,1.03,1.06,1.05,1.01,1.0,1.0,1.02,1.15,1.36,TL50_FL81,13,0,13,0.0,13.0,0.0,0.0,46.15,7.69,0.0


In [10]:
df_bund_p1_tall.head(200)

,Method,Week 4,Week 8,Week 12,Week 16,Week 20,Week 24,Week 28,Week 32,Week 36,Total,Method2,Size,Max0,Max1,Aver_0,Aver_1,STD_0,STD_1,ODD_12,ODD_15,ODD_2
0,TL92_FL7,1.11,1.82,1.06,1.34,1.05,1.0,1.0,0.74,1.06,2.36,TL92_FL7,34,3,10,1.1,3.7,4.0,17.0,73.53,35.29,14.71
1,TL93_FL6,1.11,1.82,1.06,1.34,1.05,1.0,1.0,0.71,1.10,2.32,TL93_FL6,34,3,10,1.1,3.7,4.0,17.0,73.53,32.35,14.71
2,TL87_FL8,1.00,1.50,1.06,1.20,1.25,1.0,1.0,0.67,1.37,2.19,TL87_FL8,50,3,13,1.4,3.6,8.0,30.0,82.00,48.00,18.00
3,TL90_FL7,1.00,1.84,1.06,1.08,1.25,1.0,1.0,0.67,1.18,2.08,TL90_FL7,45,3,9,1.3,3.2,6.0,21.0,80.00,46.67,15.56
4,TL84_FL13,1.00,1.52,1.06,0.98,1.18,1.0,1.0,0.74,1.48,2.04,TL84_FL13,43,3,11,1.4,4.0,6.0,24.0,79.07,41.86,16.28
5,TL91_FL6,1.00,1.84,1.06,0.98,1.18,1.0,1.0,0.67,1.36,2.03,TL91_FL6,46,3,9,1.3,2.9,6.0,21.0,80.43,47.83,19.57
6,TL91_FL7,1.00,1.64,1.06,1.20,1.18,1.0,1.0,0.74,1.10,2.00,TL91_FL7,39,3,10,1.4,3.5,6.0,20.0,76.92,41.03,17.95
7,TL86_FL9,1.00,1.52,1.06,0.98,1.25,1.0,1.0,0.67,1.48,1.95,TL86_FL9,45,3,11,1.3,3.7,6.0,26.0,80.00,44.44,15.56
8,TL85_FL8,1.00,1.59,1.06,1.13,1.01,1.0,1.0,0.73,1.37,1.91,TL85_FL8,57,3,12,1.3,3.1,8.0,27.0,84.21,49.12,17.54
9,TL89_FL7,1.00,1.73,1.06,1.08,1.25,1.0,1.0,0.67,1.15,1.90,TL89_FL7,49,3,10,1.4,3.1,8.0,23.0,81.63,48.98,16.33


In [378]:
df_bund_p1_tall.loc[df_bund_p1_tall['Method']=='TL92_FL7']

,Method,Week 4,Week 8,Week 12,Week 16,Week 20,Week 24,Week 28,Week 32,Week 36,Total,Method2,Size,Max0,Max1,Aver_0,Aver_1,STD_0,STD_1,ODD_12,ODD_15,ODD_2
0,TL92_FL7,1.11,1.82,1.06,1.34,1.05,1.0,1.0,0.74,1.06,2.36,TL92_FL7,34,3,10,1.1,3.7,4.0,17.0,73.53,35.29,14.71


In [394]:
df_bund_p1.sort_values('Week 4',ascending=False).head(10)

,Method,Week 4,Week 8,Week 12,Week 16,Week 20,Week 24,Week 28,Week 32,Total,Method2,Size,Max0,Max1,Aver_0,Aver_1,STD_0,STD_1,ODD_12,ODD_15,ODD_2
208,TL61_FL37,1.21,1.02,1.06,0.94,1.24,1.0,1.0,0.95,1.45,TL61_FL37,32,3,13,1.5,8.7,4.0,9.0,81.25,34.38,3.12
262,TL59_FL43,1.21,1.02,1.06,0.85,1.24,1.0,1.0,0.95,1.31,TL59_FL43,31,3,12,1.5,8.3,4.0,9.0,80.65,32.26,0.00
264,TL57_FL47,1.21,1.02,1.06,0.85,1.24,1.0,1.0,0.95,1.31,TL57_FL47,31,3,12,1.5,8.3,4.0,9.0,80.65,32.26,0.00
265,TL60_FL40,1.21,1.02,1.06,0.85,1.24,1.0,1.0,0.95,1.31,TL60_FL40,31,3,12,1.5,8.3,4.0,9.0,80.65,32.26,0.00
266,TL59_FL42,1.21,1.02,1.06,0.85,1.24,1.0,1.0,0.95,1.31,TL59_FL42,31,3,12,1.5,8.3,4.0,9.0,80.65,32.26,0.00
267,TL57_FL49,1.21,1.02,1.06,0.85,1.24,1.0,1.0,0.95,1.31,TL57_FL49,31,3,12,1.5,8.3,4.0,9.0,80.65,32.26,0.00
280,TL55_FL54,1.21,0.98,1.06,0.83,1.29,1.0,1.0,0.95,1.27,TL55_FL54,29,3,11,1.5,7.7,4.0,7.0,79.31,34.48,3.45
520,TL57_FL51,1.21,0.98,1.06,0.83,1.15,1.0,1.0,0.95,1.13,TL57_FL51,28,3,11,1.5,7.3,4.0,7.0,78.57,32.14,0.00
526,TL55_FL55,1.21,0.98,1.06,0.83,1.15,1.0,1.0,0.95,1.13,TL55_FL55,28,3,11,1.5,7.3,4.0,7.0,78.57,32.14,0.00
528,TL56_FL53,1.21,0.98,1.06,0.83,1.15,1.0,1.0,0.95,1.13,TL56_FL53,28,3,11,1.5,7.3,4.0,7.0,78.57,32.14,0.00


In [47]:
df_bund_p1_t2.head(50)

,Method,Week 4,Week 8,Total,Method2,Size,Max0,Max1,Aver_0,Aver_1,STD_0,STD_1,ODD_12,ODD_15,ODD_2
0,TL56_FL71,1.06,1.14,1.21,TL56_FL71,6,0,6,0.0,6.0,0.0,0.0,66.67,16.67,0.0
1,TL52_FL73,1.06,1.14,1.21,TL52_FL73,6,0,6,0.0,6.0,0.0,0.0,66.67,16.67,0.0
2,TL51_FL75,1.06,1.14,1.21,TL51_FL75,6,0,6,0.0,6.0,0.0,0.0,66.67,16.67,0.0
3,TL51_FL74,1.06,1.14,1.21,TL51_FL74,6,0,6,0.0,6.0,0.0,0.0,66.67,16.67,0.0
4,TL51_FL73,1.06,1.14,1.21,TL51_FL73,6,0,6,0.0,6.0,0.0,0.0,66.67,16.67,0.0
5,TL56_FL68,1.06,1.14,1.21,TL56_FL68,6,0,6,0.0,6.0,0.0,0.0,66.67,16.67,0.0
6,TL53_FL70,1.06,1.14,1.21,TL53_FL70,6,0,6,0.0,6.0,0.0,0.0,66.67,16.67,0.0
7,TL53_FL71,1.06,1.14,1.21,TL53_FL71,6,0,6,0.0,6.0,0.0,0.0,66.67,16.67,0.0
8,TL53_FL72,1.06,1.14,1.21,TL53_FL72,6,0,6,0.0,6.0,0.0,0.0,66.67,16.67,0.0
9,TL53_FL73,1.06,1.14,1.21,TL53_FL73,6,0,6,0.0,6.0,0.0,0.0,66.67,16.67,0.0


# SA

In [66]:
#df6_sa_p2_s = Analyse_Model(repos6_SA_P2_S)
#df6_sa_p2_s.loc[df6_sa_p2_s['Method']=='TL50_FL25']
df6_sa_p2_s.head(10)

,Method,Week 4,Week 8,Week 12,Week 16,Week 20,Week 24,Week 28,Week 32,Total,Method2,Size,Max0,Max1,Aver_0,Aver_1,STD_0,STD_1,ODD_12,ODD_15,ODD_2
0,TL57_FL46,0.97,0.97,1.08,1.0,1.0,1.0,1.61,1.27,2.04,TL57_FL46,28,2,18,1.0,6.0,2.0,24.0,85.71,42.86,7.14
1,TL53_FL49,0.97,0.97,1.08,1.0,1.0,1.0,1.51,1.26,1.92,TL53_FL49,30,2,14,1.0,5.0,2.0,18.0,86.67,43.33,10.00
2,TL52_FL51,0.97,1.01,1.08,1.0,1.0,1.0,1.47,1.23,1.90,TL52_FL51,24,2,16,1.0,7.0,2.0,18.0,83.33,37.50,4.17
3,TL51_FL52,0.97,1.01,1.08,1.0,1.0,1.0,1.47,1.23,1.90,TL51_FL52,24,2,16,1.0,7.0,2.0,18.0,83.33,37.50,4.17
4,TL53_FL50,0.97,1.07,1.08,1.0,1.0,1.0,1.35,1.26,1.90,TL53_FL50,25,2,17,1.0,7.3,2.0,19.0,84.00,36.00,4.00
5,TL50_FL54,0.97,1.01,1.08,1.0,1.0,1.0,1.47,1.23,1.90,TL50_FL54,24,2,16,1.0,7.0,2.0,18.0,83.33,37.50,4.17
6,TL54_FL49,0.97,1.07,1.08,1.0,1.0,1.0,1.47,1.15,1.88,TL54_FL49,25,2,17,1.0,7.3,2.0,19.0,84.00,36.00,0.00
7,TL56_FL47,0.97,0.97,1.08,1.0,1.0,1.0,1.47,1.26,1.87,TL56_FL47,27,2,17,1.0,5.8,2.0,22.0,85.19,40.74,7.41
8,TL55_FL48,0.97,0.97,1.08,1.0,1.0,1.0,1.47,1.27,1.86,TL55_FL48,27,2,17,1.0,5.8,2.0,22.0,85.19,40.74,7.41
9,TL50_FL53,0.97,0.97,1.08,1.0,1.0,1.0,1.61,1.14,1.84,TL50_FL53,29,2,17,1.0,4.8,2.0,24.0,86.21,44.83,10.34


In [461]:
#df3_sa_p1=Analyse_Model(repos3_SA_P1)

In [337]:
df3_sa_p1.head(10)

,Method,Week 4,Week 8,Week 12,Week 16,Week 20,Week 24,Total,Method2,Size,Max0,Max1,Aver_0,Aver_1,STD_0,STD_1,ODD_12,ODD_15,ODD_2
0,TL89_FL6,1.20,0.98,1.11,1.02,1.14,1.08,1.63,TL89_FL6,30,1,6,1.0,3.4,0.0,11.0,96.67,26.67,6.67
1,TL89_FL7,1.20,0.98,1.11,1.02,1.14,1.08,1.63,TL89_FL7,30,1,6,1.0,3.4,0.0,11.0,96.67,26.67,6.67
2,TL83_FL10,1.11,1.20,1.14,0.84,1.16,1.08,1.60,TL83_FL10,36,2,7,1.1,3.5,2.0,16.0,94.44,22.22,8.33
3,TL84_FL9,1.11,1.08,1.14,0.93,1.16,1.08,1.60,TL84_FL9,36,1,5,1.0,3.1,0.0,15.0,94.44,25.00,8.33
4,TL90_FL6,1.20,0.98,1.11,1.02,1.10,1.08,1.58,TL90_FL6,29,1,6,1.0,3.3,0.0,10.0,96.55,27.59,6.90
5,TL85_FL8,1.11,1.08,1.03,1.02,1.16,1.08,1.57,TL85_FL8,38,1,6,1.0,2.9,0.0,15.0,94.74,28.95,7.89
6,TL88_FL7,1.11,1.02,1.11,1.02,1.14,1.08,1.56,TL88_FL7,33,1,6,1.0,3.2,0.0,14.0,96.97,24.24,6.06
7,TL87_FL7,1.11,1.02,1.11,1.02,1.14,1.08,1.56,TL87_FL7,33,1,6,1.0,3.2,0.0,14.0,96.97,24.24,6.06
8,TL85_FL13,1.23,0.95,1.24,0.93,1.07,1.08,1.55,TL85_FL13,27,1,9,1.0,3.7,0.0,13.0,92.59,22.22,7.41
9,TL80_FL21,1.23,0.95,1.24,0.93,1.07,1.08,1.55,TL80_FL21,27,1,9,1.0,3.7,0.0,13.0,92.59,22.22,7.41


In [21]:
df_sa_p2_s=Analyse_Model(repos_SA_P2_S)

TL50_FL1 weekly_profit
TL60_FL1 weekly_profit
TL70_FL1 weekly_profit
TL80_FL1 weekly_profit
TL90_FL1 weekly_profit
TL100_FL1 weekly_profit
TL50_FL1 weekly_profit
TL60_FL1 weekly_profit
TL70_FL1 weekly_profit
TL80_FL1 weekly_profit
TL90_FL1 weekly_profit
TL100_FL1 weekly_profit
TL50_FL1 weekly_profit
TL60_FL1 weekly_profit
TL70_FL1 weekly_profit
TL80_FL1 weekly_profit
TL90_FL1 weekly_profit
TL100_FL1 weekly_profit
TL50_FL1 weekly_profit
TL60_FL1 weekly_profit
TL70_FL1 weekly_profit
TL80_FL1 weekly_profit
TL90_FL1 weekly_profit
TL100_FL1 weekly_profit
TL50_FL1 weekly_profit
TL60_FL1 weekly_profit
TL70_FL1 weekly_profit
TL80_FL1 weekly_profit
TL90_FL1 weekly_profit
TL100_FL1 weekly_profit
TL50_FL1 weekly_profit
TL60_FL1 weekly_profit
TL70_FL1 weekly_profit
TL80_FL1 weekly_profit
TL90_FL1 weekly_profit
TL100_FL1 weekly_profit
TL50_FL1 weekly_profit
TL60_FL1 weekly_profit
TL70_FL1 weekly_profit
TL80_FL1 weekly_profit
TL90_FL1 weekly_profit
TL100_FL1 weekly_profit
TL50_FL1 model_std
TL60_FL1

In [96]:
df_SA = go_get(repos_SA_P2_S,mode=2,lista=['Tree_Forest','TL56','FL55','Predicted'])

In [23]:
df_sa_p2_s.head(20)

,Method,Week 4,Week 8,Week 12,Week 16,Week 20,Week 24,Total,Method2,Size,Max0,Max1,Aver_0,Aver_1,STD_0,STD_1,ODD_12,ODD_15,ODD_2
0,TL56_FL55,1.21,1.13,1.00,1.03,1.13,1.08,1.72,TL56_FL55,26,1,11,0.8,5.8,2.0,10.0,92.31,19.23,0.00
1,TL55_FL56,1.26,1.13,1.00,0.93,1.19,1.08,1.71,TL55_FL56,26,1,12,0.8,5.8,2.0,12.0,92.31,19.23,0.00
2,TL61_FL49,1.26,1.02,0.98,1.03,1.19,1.08,1.66,TL61_FL49,28,1,7,0.8,4.8,2.0,5.0,92.86,21.43,0.00
3,TL56_FL56,1.21,1.09,0.98,1.04,1.13,1.08,1.63,TL56_FL56,21,1,9,0.7,6.3,1.0,5.0,90.48,19.05,0.00
4,TL58_FL54,1.21,1.09,0.98,1.03,1.13,1.08,1.63,TL58_FL54,21,1,9,0.7,6.3,1.0,7.0,90.48,19.05,0.00
5,TL61_FL50,1.26,1.02,0.98,1.00,1.19,1.08,1.61,TL61_FL50,27,1,7,0.8,4.6,2.0,6.0,92.59,22.22,0.00
6,TL62_FL49,1.26,0.98,0.98,1.03,1.19,1.08,1.61,TL62_FL49,27,1,7,0.8,4.6,2.0,6.0,92.59,22.22,0.00
7,TL52_FL57,1.17,1.02,0.93,1.03,1.29,1.08,1.59,TL52_FL57,34,1,8,1.0,4.0,0.0,16.0,94.12,23.53,2.94
8,TL55_FL57,1.26,1.09,1.00,0.93,1.13,1.08,1.57,TL55_FL57,24,1,11,0.8,5.2,2.0,12.0,91.67,16.67,0.00
9,TL59_FL54,1.21,1.09,0.98,1.00,1.13,1.08,1.57,TL59_FL54,20,1,9,0.7,6.0,1.0,6.0,90.00,20.00,0.00


In [25]:
Aux_All1(df_sa_p2_s).head(20)

,Method,Week 4,Week 8,Week 12,Week 16,Week 20,Week 24,Total,Method2,Size,Max0,Max1,Aver_0,Aver_1,STD_0,STD_1,ODD_12,ODD_15,ODD_2
0,TL56_FL55,1.21,1.13,1.00,1.03,1.13,1.08,1.72,TL56_FL55,26,1,11,0.8,5.8,2.0,10.0,92.31,19.23,0.0
1,TL57_FL52,1.26,1.02,1.00,1.03,1.07,1.08,1.53,TL57_FL52,30,2,7,1.0,5.0,2.0,4.0,93.33,23.33,0.0
2,TL55_FL55,1.02,1.02,1.00,1.03,1.19,1.08,1.38,TL55_FL55,31,1,5,1.0,3.6,0.0,8.0,93.55,19.35,0.0
3,TL96_FL14,1.06,1.03,1.06,1.02,1.02,1.08,1.29,TL96_FL14,10,1,7,0.5,4.5,1.0,5.0,90.00,20.00,10.0
4,TL85_FL32,1.03,1.03,1.06,1.00,1.02,1.08,1.23,TL85_FL32,6,0,6,0.0,6.0,0.0,0.0,83.33,16.67,0.0
5,TL88_FL28,1.03,1.03,1.06,1.00,1.02,1.08,1.23,TL88_FL28,6,0,6,0.0,6.0,0.0,0.0,83.33,16.67,0.0
6,TL87_FL30,1.03,1.03,1.06,1.00,1.02,1.08,1.23,TL87_FL30,6,0,6,0.0,6.0,0.0,0.0,83.33,16.67,0.0
7,TL86_FL31,1.03,1.03,1.06,1.00,1.02,1.08,1.23,TL86_FL31,6,0,6,0.0,6.0,0.0,0.0,83.33,16.67,0.0
8,TL68_FL59,1.00,1.06,1.00,1.00,1.05,1.08,1.19,TL68_FL59,4,0,4,0.0,4.0,0.0,0.0,100.00,25.00,0.0
9,TL66_FL60,1.00,1.06,1.00,1.00,1.05,1.08,1.19,TL66_FL60,4,0,4,0.0,4.0,0.0,0.0,100.00,25.00,0.0


# L

In [471]:
#df6_l_p2_s_20 = Analyse_Model(repos6_L_P2_S_20)
#df6_l_p2_s_20

In [65]:
#df6_l_p2_s = Analyse_Model(repos6_L_P2_S)
Aux_All1(df6_l_p2_s,All=0.7).head(20)

,Method,Week 4,Week 8,Week 12,Week 16,Week 20,Week 24,Total,Method2,Size,Max0,Max1,Aver_0,Aver_1,STD_0,STD_1,ODD_12,ODD_15,ODD_2
0,TL83_FL16,1.51,1.13,1.27,0.80,0.79,1.0,1.37,TL83_FL16,43,5,10,1.9,3.5,7.0,23.0,88.37,72.09,34.88
1,TL54_FL62,1.02,1.12,1.04,1.01,1.12,1.0,1.35,TL54_FL62,10,0,10,0.0,10.0,0.0,0.0,60.00,20.00,0.00
2,TL55_FL61,1.02,1.12,1.04,1.01,1.12,1.0,1.35,TL55_FL61,10,0,10,0.0,10.0,0.0,0.0,60.00,20.00,0.00
3,TL52_FL66,1.02,1.12,1.04,1.01,1.12,1.0,1.35,TL52_FL66,10,0,10,0.0,10.0,0.0,0.0,60.00,20.00,0.00
4,TL73_FL31,1.02,1.23,1.27,0.91,0.92,1.0,1.33,TL73_FL31,24,2,8,1.2,3.6,2.0,10.0,79.17,50.00,37.50
5,TL50_FL70,1.02,1.10,1.04,1.01,1.12,1.0,1.32,TL50_FL70,9,0,9,0.0,9.0,0.0,0.0,55.56,22.22,0.00
6,TL50_FL69,1.02,1.10,1.04,1.01,1.12,1.0,1.32,TL50_FL69,9,0,9,0.0,9.0,0.0,0.0,55.56,22.22,0.00
7,TL51_FL68,1.02,1.10,1.04,1.01,1.12,1.0,1.32,TL51_FL68,9,0,9,0.0,9.0,0.0,0.0,55.56,22.22,0.00
8,TL82_FL16,1.51,1.13,1.27,0.80,0.76,1.0,1.31,TL82_FL16,45,5,10,2.0,3.2,8.0,25.0,88.89,73.33,33.33
9,TL90_FL11,1.40,1.18,1.27,0.74,0.83,1.0,1.29,TL90_FL11,35,4,9,1.6,3.4,6.0,18.0,85.71,65.71,28.57


In [363]:
#g2df5_l_p2_s = Analyse_Model(G2repos5_L_P2_S)
Aux_All1(g2df5_l_p2_s,All=0.7).head(20)

,Method,Week 4,Week 8,Week 12,Week 16,Week 20,Week 24,Total,Method2,Size,Max0,Max1,Aver_0,Aver_1,STD_0,STD_1,ODD_12,ODD_15,ODD_2
0,TL67_FL34,1.83,1.98,1.27,0.74,0.79,1.11,2.96,TL67_FL34,48,5,16,2.0,6.0,8.0,30.0,89.58,66.67,22.92
1,TL79_FL18,1.64,2.12,1.27,0.82,0.79,1.00,2.85,TL79_FL18,48,4,16,2.0,6.0,6.0,32.0,89.58,66.67,18.75
2,TL66_FL36,1.75,1.84,1.27,0.74,0.79,1.11,2.64,TL66_FL36,46,5,15,2.0,5.7,8.0,27.0,89.13,67.39,23.91
3,TL78_FL20,1.64,2.12,1.27,0.74,0.79,1.00,2.56,TL78_FL20,49,5,16,2.2,6.0,7.0,32.0,89.80,67.35,20.41
4,TL79_FL19,1.65,1.89,1.27,0.82,0.79,1.00,2.54,TL79_FL19,44,4,14,1.8,6.6,7.0,24.0,88.64,63.64,18.18
5,TL79_FL17,1.48,2.12,1.27,0.75,0.71,1.11,2.35,TL79_FL17,54,5,16,2.0,4.2,10.0,33.0,90.74,70.37,25.93
6,TL78_FL21,1.49,1.89,1.27,0.82,0.79,1.00,2.29,TL78_FL21,45,4,14,2.0,5.5,6.0,28.0,88.89,64.44,17.78
7,TL81_FL17,1.49,1.89,1.27,0.82,0.79,1.00,2.29,TL81_FL17,45,4,14,2.0,5.5,6.0,28.0,88.89,64.44,17.78
8,TL81_FL16,1.64,1.89,1.27,0.74,0.79,1.00,2.28,TL81_FL16,47,5,14,2.2,5.7,7.0,29.0,89.36,65.96,21.28
9,TL55_FL54,1.34,1.76,1.14,0.76,0.97,1.11,2.22,TL55_FL54,36,3,12,1.3,4.7,5.0,20.0,86.11,58.33,16.67


In [57]:
df_L = go_get(repos_L_P2,mode=2,lista=['Tree_Forest','TL70','FL25','ALL'])

In [58]:
df_L.sort_values('Tree Score %',ascending=False)

,Index,League,Date,Time,HT,AT,Tree Score %,Tree_var%70,ForestPredix_25%,ForestReal_25%,Won/NotWon,ODDH_Aver.,ODDD_Aver.,ODDA_Aver.
36,36,French Ligue 1,2019-11-22,1945,Paris Saint-Germain,Lille,73.48,66.77,1,YES,1,1.27,5.92,10.59
118,118,French Ligue 1,2020-02-01,1630,Paris Saint-Germain,Montpellier,73.35,66.14,1,YES,1,1.14,8.09,19.40
16,16,French Ligue 1,2019-10-27,2000,Paris Saint-Germain,Marseille,72.95,60.92,1,YES,1,1.23,6.56,11.49
63,63,French Ligue 1,2019-12-04,2005,Paris Saint-Germain,Nantes,72.46,61.39,1,YES,1,1.15,7.69,18.03
93,93,French Ligue 1,2019-12-21,1945,Marseille,Nimes,72.45,63.77,1,YES,1,1.28,5.69,10.19
146,146,French Ligue 1,2020-02-09,2005,Paris Saint-Germain,Lyon,72.40,64.72,1,YES,1,1.27,6.25,9.49
91,91,French Ligue 1,2019-12-21,1945,Paris Saint-Germain,Amiens,70.29,60.28,1,YES,1,1.07,11.65,28.83
168,168,French Ligue 1,2020-02-29,1630,Paris Saint-Germain,Dijon,70.14,61.79,1,YES,1,1.13,9.12,17.52
166,166,French Ligue 1,2020-02-23,2000,Paris Saint-Germain,Bordeaux,66.04,50.47,1,YES,1,1.13,8.43,18.52
132,132,French Ligue 1,2020-02-05,1800,Lyon,Amiens,64.06,45.97,1,NO,0,1.32,5.13,9.65


In [60]:
#df_l_p2 = Analyse_Model(repos_L_P2)
#df_l_p2.head(10)
Aux_All1(df_l_p2,All=0.9).head(100)
#df_l_p2.loc[df_l_p2['Max0']>=1].sort_values(['Max0','Total','Size','ODD_2'],ascending=[True,False,True,False])
#df_L = go_get(repos_L_P2,mode=2,lista=['Tree_Forest','TL70','FL24','Predicted'])

,Method,Week 4,Week 8,Week 12,Week 16,Week 20,Week 24,Total,Method2,Size,Max0,Max1,Aver_0,Aver_1,STD_0,STD_1,ODD_12,ODD_15,ODD_2
0,TL98_FL6,2.27,1.06,1.04,0.93,1.18,1.23,3.37,TL98_FL6,21,2,8,1.0,4.2,2.0,9.0,76.19,42.86,28.57
1,TL99_FL5,2.04,1.20,1.04,1.01,1.02,1.11,2.91,TL99_FL5,27,3,5,1.1,2.6,4.0,9.0,81.48,62.96,44.44
2,TL70_FL24,1.21,1.94,1.22,0.93,1.10,1.00,2.91,TL70_FL24,34,2,15,1.0,5.8,2.0,18.0,85.29,52.94,8.82
3,TL100_FL5,2.04,1.20,1.04,1.01,1.02,1.11,2.91,TL100_FL5,27,3,5,1.1,2.6,4.0,9.0,81.48,62.96,44.44
4,TL68_FL27,1.34,1.94,1.10,0.93,1.06,1.00,2.82,TL68_FL27,31,2,19,1.0,6.8,2.0,24.0,83.87,51.61,6.45
5,TL67_FL27,1.34,1.94,1.10,0.93,1.06,1.00,2.82,TL67_FL27,31,2,19,1.0,6.8,2.0,24.0,83.87,51.61,6.45
6,TL76_FL19,1.21,1.94,1.10,0.93,1.10,1.00,2.63,TL76_FL19,33,2,14,1.0,5.6,2.0,17.0,84.85,51.52,6.06
7,TL75_FL20,1.21,1.94,1.10,0.93,1.10,1.00,2.63,TL75_FL20,33,2,14,1.0,5.6,2.0,17.0,84.85,51.52,6.06
8,TL73_FL22,1.21,1.94,1.10,0.93,1.10,1.00,2.63,TL73_FL22,33,2,14,1.0,5.6,2.0,17.0,84.85,51.52,6.06
9,TL71_FL24,1.21,1.94,1.10,0.93,1.10,1.00,2.63,TL71_FL24,33,2,14,1.0,5.6,2.0,17.0,84.85,51.52,6.06


In [19]:
#df6_l_p2_s_20 = Analyse_Model(repos6_L_P2_S_20)
Aux_All1(df6_l_p2_s_20,All=0.6).head(10)

,Method,Week 4,Week 8,Week 12,Week 16,Week 20,Week 24,Week 28,Week 32,Week 36,Week 40,Week 44,Week 48,Week 52,Week 56,Total,Method2,Size,Max0,Max1,Aver_0,Aver_1,STD_0,STD_1,ODD_12,ODD_15,ODD_2
0,TL81_FL16,1.48,0.71,1.19,1.27,1.0,1.0,1.0,0.77,1.03,3.38,1.61,0.77,0.72,1.05,4.03,TL81_FL16,214,6,9,2.0,2.5,45.0,75.0,96.26,82.71,44.39
1,TL86_FL14,1.48,0.98,1.10,1.03,1.0,1.0,1.0,0.93,1.04,2.26,1.99,0.96,0.62,0.70,3.00,TL86_FL14,193,8,9,1.9,2.4,44.0,71.0,95.85,81.35,40.93
2,TL83_FL15,1.33,0.71,0.89,1.03,1.0,1.0,1.0,0.93,1.03,3.44,1.79,0.77,0.72,0.97,2.80,TL83_FL15,211,5,9,2.0,2.5,46.0,74.0,96.21,82.46,44.08
3,TL88_FL13,1.32,0.88,1.10,1.15,1.0,1.0,1.0,0.84,0.84,2.26,2.05,0.96,0.62,0.63,1.79,TL88_FL13,191,8,14,2.0,2.5,39.0,71.0,95.81,81.15,41.36
4,TL81_FL20,1.26,1.09,0.81,1.09,1.0,1.0,1.0,0.95,0.87,1.29,1.75,1.36,0.68,0.67,1.40,TL81_FL20,121,6,11,1.8,2.8,26.0,36.0,93.39,74.38,32.23
5,TL87_FL13,1.48,0.71,1.07,1.03,1.0,1.0,1.0,0.76,0.92,2.26,1.79,0.96,0.62,0.70,1.38,TL87_FL13,203,8,9,2.0,2.4,44.0,74.0,96.06,82.27,41.87
6,TL85_FL14,1.48,0.71,0.87,1.03,1.0,1.0,1.0,0.76,0.93,2.91,1.61,0.86,0.72,0.69,1.34,TL85_FL14,217,6,9,2.0,2.3,48.0,74.0,96.31,82.95,44.70
7,TL100_FL8,1.20,0.85,0.81,1.27,1.0,1.0,1.0,1.09,1.08,1.23,1.22,1.15,0.85,0.73,1.30,TL100_FL8,53,4,8,1.5,3.2,11.0,23.0,86.79,56.60,30.19
8,TL99_FL8,1.20,0.85,0.81,1.27,1.0,1.0,1.0,1.09,1.08,1.23,1.22,1.15,0.85,0.73,1.30,TL99_FL8,53,4,8,1.5,3.2,11.0,23.0,86.79,56.60,30.19
9,TL96_FL24,1.02,1.03,1.00,1.02,1.0,1.0,1.0,1.00,1.00,1.02,1.04,1.01,1.04,1.01,1.22,TL96_FL24,10,0,10,0.0,10.0,0.0,0.0,60.00,0.00,0.00


In [99]:
#df6_l_p2_s_20_opt1=Analyse_Model(repos6_L_P2_S_20_opt1)
df6_l_p2_s_20_opt1.head(10)
#df6_l_p2_s_20_opt1.loc[df6_l_p2_s_20_opt1['Method']=='TL50_FL38']

,Method,Week 4,Week 8,Week 12,Week 16,Week 20,Week 24,Week 28,Week 32,Week 36,Total,Method2,Size,Max0,Max1,Aver_0,Aver_1,STD_0,STD_1,ODD_12,ODD_15,ODD_2
0,TL85_FL5,2.03,0.72,1.85,1.54,1.0,1.0,1.0,1.09,0.71,3.25,TL85_FL5,114,6,6,1.9,2.5,30.0,34.0,97.37,85.09,44.74
1,TL79_FL7,2.05,0.93,1.41,1.36,1.0,1.0,1.0,1.21,0.71,3.16,TL79_FL7,118,6,6,1.9,2.4,26.0,34.0,97.46,85.59,44.92
2,TL84_FL6,1.79,0.89,1.20,1.47,1.0,1.0,1.0,1.35,0.79,3.03,TL84_FL6,102,6,6,1.7,2.4,22.0,31.0,97.06,83.33,40.20
3,TL80_FL8,1.79,1.03,1.20,1.25,1.0,1.0,1.0,1.22,0.88,2.99,TL80_FL8,103,5,5,1.8,2.4,22.0,31.0,97.09,83.50,41.75
4,TL82_FL7,1.79,1.15,1.08,1.25,1.0,1.0,1.0,1.22,0.88,2.99,TL82_FL7,103,5,5,1.8,2.4,20.0,31.0,97.09,83.50,41.75
5,TL80_FL7,1.81,1.03,1.08,1.67,1.0,1.0,1.0,1.21,0.71,2.93,TL80_FL7,113,6,6,1.8,2.4,25.0,33.0,97.35,84.96,44.25
6,TL76_FL7,2.01,0.98,1.41,1.22,1.0,1.0,1.0,0.96,0.88,2.88,TL76_FL7,128,6,6,2.1,2.6,30.0,35.0,97.66,86.72,47.66
7,TL78_FL7,2.05,0.93,1.41,1.36,1.0,1.0,1.0,1.09,0.71,2.85,TL78_FL7,119,6,6,2.0,2.4,27.0,34.0,97.48,85.71,45.38
8,TL90_FL4,1.99,0.89,1.74,1.32,1.0,1.0,1.0,1.21,0.57,2.83,TL90_FL4,105,6,5,1.7,2.4,24.0,29.0,97.14,83.81,41.90
9,TL78_FL6,1.85,0.98,1.50,1.22,1.0,1.0,1.0,0.96,0.88,2.81,TL78_FL6,129,6,6,2.2,2.6,29.0,35.0,97.67,86.82,48.84


In [101]:
#df6_l_p2_s_20_opt2=Analyse_Model(repos6_L_P2_S_20_opt2)
df6_l_p2_s_20_opt2.head(10)
#df6_l_p2_s_20_opt2.loc[df6_l_p2_s_20_opt2['Method']=='TL80_FL8']

,Method,Week 4,Week 8,Week 12,Week 16,Week 20,Week 24,Total,Method2,Size,Max0,Max1,Aver_0,Aver_1,STD_0,STD_1,ODD_12,ODD_15,ODD_2
0,TL50_FL38,1.58,2.43,1.15,0.59,0.90,1.21,2.84,TL50_FL38,88,4,19,1.8,3.2,13.0,49.0,94.32,77.27,35.23
1,TL51_FL34,1.58,2.86,1.15,0.69,0.66,1.21,2.84,TL51_FL34,96,5,22,2.1,3.4,15.0,53.0,94.79,79.17,37.50
2,TL50_FL37,1.58,2.86,1.28,0.59,0.66,1.21,2.71,TL50_FL37,94,5,22,1.9,3.2,16.0,55.0,94.68,78.72,36.17
3,TL51_FL35,1.58,2.62,1.04,0.71,0.73,1.21,2.69,TL51_FL35,91,5,20,1.9,3.2,15.0,50.0,94.51,78.02,36.26
4,TL56_FL29,1.46,2.86,1.04,0.71,0.72,1.21,2.68,TL56_FL29,89,4,21,1.9,3.2,12.0,51.0,94.38,77.53,35.96
5,TL54_FL30,1.28,2.86,1.15,0.76,0.65,1.21,2.52,TL54_FL30,95,5,22,1.9,3.2,13.0,48.0,94.74,78.95,37.89
6,TL54_FL31,1.46,2.62,1.04,0.71,0.72,1.21,2.45,TL54_FL31,88,5,20,1.9,3.1,14.0,49.0,94.32,77.27,36.36
7,TL57_FL28,1.46,2.86,1.04,0.64,0.72,1.21,2.41,TL57_FL28,90,4,21,1.9,3.2,13.0,51.0,94.44,77.78,36.67
8,TL52_FL32,1.42,2.86,1.15,0.57,0.73,1.21,2.38,TL52_FL32,98,5,22,2.1,3.2,17.0,50.0,94.90,79.59,39.80
9,TL53_FL32,1.42,2.86,1.15,0.64,0.65,1.21,2.34,TL53_FL32,94,5,22,1.9,3.1,15.0,49.0,94.68,78.72,38.30


In [97]:
#df6_l_p2_s_20_nopt2=Analyse_Model(repos6_L_P2_S_20_nopt2)
#df6_l_p2_s_20_nopt2.head(10)
df6_l_p2_s_20_nopt2.loc[df6_l_p2_s_20_nopt2['Method']=='TL84_FL14']

,Method,Week 4,Week 8,Week 12,Week 16,Week 20,Week 24,Total,Method2,Size,Max0,Max1,Aver_0,Aver_1,STD_0,STD_1,ODD_12,ODD_15,ODD_2
235,TL84_FL14,2.76,2.21,0.72,0.48,0.41,1.84,1.57,TL84_FL14,120,6,9,2.3,2.6,28.0,41.0,95.83,83.33,49.17


In [93]:
#df6_l_p2_s_20_nopt1=Analyse_Model(repos6_L_P2_S_20_nopt1)
df6_l_p2_s_20_nopt1.head(10)

,Method,Week 4,Week 8,Week 12,Week 16,Week 20,Week 24,Week 28,Week 32,Week 36,Total,Method2,Size,Max0,Max1,Aver_0,Aver_1,STD_0,STD_1,ODD_12,ODD_15,ODD_2
0,TL86_FL14,1.48,0.98,1.10,1.03,1.0,1.0,1.0,0.93,0.99,1.52,TL86_FL14,94,4,7,1.7,2.2,22.0,32.0,96.81,81.91,39.36
1,TL84_FL14,1.68,0.84,1.16,1.27,1.0,1.0,1.0,0.76,0.88,1.38,TL84_FL14,110,6,5,1.8,2.1,27.0,32.0,97.27,84.55,44.55
2,TL94_FL9,1.81,0.73,1.15,1.15,1.0,1.0,1.0,1.04,0.73,1.33,TL94_FL9,82,4,5,1.6,2.4,17.0,25.0,96.34,79.27,31.71
3,TL79_FL17,1.68,0.75,1.19,1.37,1.0,1.0,1.0,0.69,0.88,1.26,TL79_FL17,109,6,5,2.0,2.2,25.0,33.0,97.25,84.40,44.95
4,TL84_FL21,1.24,1.11,0.78,1.15,1.0,1.0,1.0,0.95,1.07,1.25,TL84_FL21,40,4,5,1.6,2.9,8.0,13.0,92.50,65.00,20.00
5,TL81_FL16,1.48,0.71,1.19,1.27,1.0,1.0,1.0,0.77,0.98,1.19,TL81_FL16,103,6,5,1.9,2.2,24.0,30.0,97.09,83.50,42.72
6,TL85_FL20,1.24,1.11,0.78,1.09,1.0,1.0,1.0,0.95,1.07,1.18,TL85_FL20,39,4,5,1.6,2.8,8.0,12.0,92.31,64.10,20.51
7,TL99_FL8,1.20,0.85,0.81,1.27,1.0,1.0,1.0,1.09,1.02,1.17,TL99_FL8,20,4,6,1.4,2.6,5.0,10.0,90.00,45.00,30.00
8,TL100_FL8,1.20,0.85,0.81,1.27,1.0,1.0,1.0,1.09,1.02,1.17,TL100_FL8,20,4,6,1.4,2.6,5.0,10.0,90.00,45.00,30.00
9,TL100_FL11,1.15,0.94,0.90,1.05,1.0,1.0,1.0,1.09,1.02,1.14,TL100_FL11,15,2,4,1.0,2.0,2.0,6.0,86.67,46.67,33.33


# Funções

In [15]:
def Aux_All1(AM_df,count=0,columns=None,All=1):
    #Dá os modelos que só deram profit nas "Week"
    if count==0:
        C=list(AM_df.columns)
        C = [x for x in C if 'Week' in x]
        return AM_df.loc[Aux_All1(AM_df,count=1,columns=C,All=All)].sort_values(['Total'],ascending=[False]).reset_index(drop=True)
    else:
        if len(columns)==1:
            return AM_df[columns[0]]>=All
        elif len(columns)==2:
            return np.logical_and(AM_df[columns[0]]>=All,AM_df[columns[1]]>=All)
        else:
            p1 = columns[int(len(columns)/2):]
            p2 = columns[:int(len(columns)/2)]
            return np.logical_and(Aux_All1(AM_df,count=1,columns=p1,All=All),Aux_All1(AM_df,count=1,columns=p2,All=All))

In [36]:
def money_boss(repos,odd=1):
#diz quanto gastamos em apostas com Odd inferior a 1.5
    DF = go_get(repos,mode=2,lista=['OverAll'])
    method = list(DF['Method'])
    u15_cash = [0]*len(method)
    method_cash = [100]*len(method)
    for i in range(len(method)):
        tree,foret = method[i].split('_')
        listaz = ['Tree_Forest',tree,foret,'Predicted']
        df = go_get(repos,mode=2,lista=listaz)
        df = df.loc[df['ODDH_Aver.']>odd]
        wnw = list(df['Won/NotWon'])
        for n in range(len(df)):
            bet_money = 0.1*method_cash[i]
            method_cash[i]-=bet_money
            if df.iloc[n]['ODDH_Aver.']>=1.5:
                u15_cash[i]+=bet_money
            if df.iloc[n]['Won/NotWon']==1:
                add = bet_money * df.iloc[n]['ODDH_Aver.']
                method_cash[i]+=add
    ok1 = pd.DataFrame({'Money_U15':list(map(lambda x:round(x/100,2),u15_cash))})
    return ok1

In [98]:
def bet_with_time(df,modeT=0,perc=0.1): #temos de arranjar estratégias para jogos no mesmo dia
    money=100
    limit = len(df)
    df=df.sort_values(['Date','Time'],ascending=[True,True]).reset_index(drop=True)
    d = 0
    if modeT==0: #jogos a mesma hora são transformados em apostas múltiplas (2 a 2)
        while d<len(df):
            try:
                if (df.iloc[d]['Date']==df.iloc[d+1]['Date']) and (abs(df.iloc[d]['Time']-df.iloc[d+1]['Time'])<300): #300 = 3h
                    print('Multiple Bet')
                    print(df.iloc[d]['HT'],'VS',df.iloc[d]['AT'],' and ',df.iloc[d+1]['HT'],'VS',df.iloc[d+1]['AT'])
                    print('ODD: ',(df.iloc[d]['ODDH_Aver.'])*(df.iloc[d+1]['ODDH_Aver.']))
                    oddz=df.iloc[d]['ODDH_Aver.']*df.iloc[d+1]['ODDH_Aver.']
                    bet_money = money*perc
                    money-=bet_money
                    if (df.iloc[d]['Won/NotWon']==1) and (df.iloc[d+1]['Won/NotWon']==1):
                        money = money + (bet_money*oddz)
                        print('Result: WON!!!!!!! :D')
                    else:
                        print('Result: Lost :(')
                    print('------------------------------')
                    d=d+2
                else:
                    oddz=df.iloc[d]['ODDH_Aver.']
                    bet_money = money*perc
                    money-=bet_money
                    if df.iloc[d]['Won/NotWon']==1:
                        money = money + (bet_money*oddz)
                    d+=1
                    
            except:
                oddz=df.iloc[d]['ODDH_Aver.']
                bet_money = money*perc
                money-=bet_money
                if df.iloc[d]['Won/NotWon']==1:
                    money = money + (bet_money*oddz)
                d+=1
        return round(money/100,2)
    if modeT==1: #o valor da aposta vai diminuindo ao longo do dia
        date=df.iloc[0]['Date']
        oddz=[]
        amount=[]
        wnw=[]
        moneyz=[money]
        while d<len(df):
            if df.iloc[d]['Date']==date:
                oddz+=[df.iloc[d]['ODDH_Aver.']]
                amount+=[moneyz[-1]*perc]
                moneyz+=[moneyz[-1]-amount[-1]]
                wnw+=[df.iloc[d]['Won/NotWon']]
                d+=1
            else:
                date=df.iloc[d]['Date']
                money=moneyz[-1]
                for i in range(len(oddz)):
                    if wnw[i]==1:
                        more = oddz[i]*amount[i]
                        money+=more
                moneyz=[money]
                oddz=[]
                amount=[]
                wnw=[]
        return round(money/100,2)

In [116]:
bet_with_time(df_LL,modeT=1)

3.3

In [115]:
df_LL

,Index,League,Date,Time,HT,AT,Tree Score %,Tree_var%95,ForestPredix_6%,ForestReal_6%,Won/NotWon,ODDH_Aver.,ODDD_Aver.,ODDA_Aver.
0,0,Spanish La Liga,2019-10-20,1300,Real Sociedad,Real Betis,56.12,9.02,1,YES,1,1.76,3.88,4.50
3,3,Spanish La Liga,2019-10-20,2000,Sevilla,Levante,62.57,7.58,1,YES,1,1.33,5.53,8.64
4,4,Spanish La Liga,2019-10-25,2000,Villarreal,Alaves,66.46,15.16,1,YES,1,1.52,4.25,6.58
5,5,Spanish La Liga,2019-10-26,1500,Leganes,Real Mallorca,63.81,12.09,1,YES,1,1.85,3.39,4.79
10,10,Spanish La Liga,2019-10-27,1500,Levante,Espanyol,55.30,7.99,1,NO,0,2.19,3.40,3.38
11,11,Spanish La Liga,2019-10-27,1730,Sevilla,Getafe,58.18,9.22,1,YES,1,1.73,3.62,5.22
14,14,Spanish La Liga,2019-10-29,2015,Barcelona,Real Valladolid,66.06,8.20,1,YES,1,1.13,9.03,19.44
16,16,Spanish La Liga,2019-10-30,1800,Real Sociedad,Levante,67.60,13.93,1,NO,0,1.50,4.55,6.21
19,19,Spanish La Liga,2019-10-30,2015,Real Madrid,Leganes,62.46,9.63,1,YES,1,1.27,5.87,11.50
24,24,Spanish La Liga,2019-11-02,1500,Levante,Barcelona,43.67,6.56,1,YES,1,8.36,5.92,1.32


In [117]:
def model_std(repos,odd=1): #devolve a maior sequência d 0s e 1s, e a std de 0s e 1s
    STD = pd.DataFrame({})
    DF = go_get(repos,mode=2,lista=['OverAll'])
    method = list(DF['Method'])
    for i in range(len(method)):
        if i%1000==0:
            print(method[i],'model_std')
        tree,foret = method[i].split('_')
        listaz = ['Tree_Forest',tree,foret,'Predicted']
        df = go_get(repos,mode=2,lista=listaz)
        df = df.loc[df['ODDH_Aver.']>odd]
        wnw = list(df['Won/NotWon'])
        lista_1 = []
        lista_0 = []
        current = 0
        ODDS=[]
        for n in range(len(wnw)):
            ODDS+=[df.iloc[n]['ODDH_Aver.']]
            if n==0:
                if wnw[n]==0:
                    lista_0+=[1]
                    current=0
                else:
                    lista_1+=[1]
                    current=1
            else:
                if current==0:
                    if wnw[n]==0:
                        lista_0[-1]+=1
                    else:
                        current=1
                        lista_1+=[1]
                elif current==1:
                    if wnw[n]==0:
                        current=0
                        lista_0+=[1]
                    else:
                        lista_1[-1]+=1
        if len(ODDS)>0:
            ODD_12 = round((sum(np.array(ODDS)>1.2)/len(ODDS))*100,2) #Percentagem de ODDS >= 1.2
            ODD_15 = round((sum(np.array(ODDS)>1.5)/len(ODDS))*100,2) #Percentagem de ODDS >= 1.5
            ODD_2 = round((sum(np.array(ODDS)>2)/len(ODDS))*100,2) #Percentagem de ODDS >= 2
        else:
            ODD_12 = 0
            ODD_15 = 0
            ODD_2 = 0
        
        
        Games_Lost = sum(lista_0)
        Max0 = max(lista_0)
        Max1 = max(lista_1)
        aver0 = np.mean(lista_0)
        aver1 = np.mean(lista_1)
        std0 = round(sum(list(map(lambda x:abs(x),list(map(lambda x:x-aver0,lista_0))))))
        std1 = round(sum(list(map(lambda x:abs(x),list(map(lambda x:x-aver1,lista_1))))))
        ok1 = pd.DataFrame({'Method2':[tree+'_'+foret],'Size':[len(wnw)],'Games_Lost(0)':[Games_Lost],'Max0':[Max0],'Max1':[Max1],
                            'Aver_0':[round(aver0,1)],'Aver_1':[round(aver1,1)],
                            #'Count_0':[len(lista_0)],'Count_1':[len(lista_1)],
                           'STD_0':[std0],'STD_1':[std1],'ODD_12':[ODD_12],'ODD_15':[ODD_15],'ODD_2':[ODD_2]})
        STD = pd.concat([STD,ok1],axis=0,sort=False)
    return STD

In [395]:
def bet_multiple(df,Max=3,Days=0,Prints=False,Multiple_Only=False,Perc=0.1):
#esta função só serve para calcular o profit de uma estratégia com apostas múltiplas
#Uma estratégia q dá bastante jeito para fugir ao controlo de apostas da Betano
#Difere do bet_with_time, pq podemos fazer apostas múltiplas com vários jogos (não só apenas 2)
#Max -> Número máximo de jogos por aposta múltipla
#Days -> intervalo de dias que os jogos das apostas múltiplas podem ter sabendo que:
# Days=0 (jogos apenas no mesmo dia com a regra das 3h) // Days=1 (jogos no mesmo dia) // e assim por diante
# Prints -> faz print das apostas múltiplas
# Multiple_Only -> Só fazemos apostas múltiplas, nada de apostas simples 
# COm Multiple_Only dps não vamos poder ver quanto dinheiro iríamos gastar em apostas simples/combinadas
    money = 100
    perc = Perc
    d=0 
    while d<len(df):
        if Days==0:
            same_games = df.loc[np.logical_and(df.iloc[d]['Date']==df['Date'],(abs(df.iloc[d]['Time']-df['Time'])<3000))] 
        else:
            same_games = df.loc[np.array(list(map(lambda x:x.days,list(abs(df.iloc[d]['Date']-df['Date'])))))<Days]
        if len(same_games)==1:
            if Multiple_Only: #assim só regista apostas múltiplas
                d+=1
                continue
            oddz = df.iloc[d]['ODDH_Aver.']
            bet_money = round(money*perc,2)
            money = money - bet_money
            if df.iloc[d]['Won/NotWon']==1:
                money = money + (bet_money*oddz)
            d+=1
        else:
            size = len(same_games)
            size_aux = len(same_games) #vai dar jeito
            if size<=Max:
                oddz = round(np.prod(list(same_games['ODDH_Aver.'])),2)
                bet_money = round(money*perc,2)
                money = money - bet_money
                if Prints:
                    print('-----------------------------------')
                    for i in range(len(same_games)):
                        print(same_games.iloc[i]['HT'],'Vs',same_games.iloc[i]['AT'],'  ODD: ',same_games.iloc[i]['ODDH_Aver.'])
                    if all(list(same_games['Won/NotWon'])):
                        print('Final Odd: ',round(np.prod(list(same_games['ODDH_Aver.'])),2),' Won!!!!!!!!!!!! :D')
                    else:
                        print('Final Odd: ',round(np.prod(list(same_games['ODDH_Aver.'])),2),' Lost :(')
                    print('-----------------------------------')
                if all(list(same_games['Won/NotWon'])):
                    money = money + (bet_money*oddz)
                d+=size
            else:
                helper = 0
                while size_aux>Max:
                    oddz = round(np.prod(list(same_games[helper:(Max+helper)]['ODDH_Aver.'])),2)
                    bet_money = round(money*perc,2)
                    money = money - bet_money
                    if Prints:
                        print('-----------------------------------')
                        for i in range(len(same_games[helper:(Max+helper)])):
                            print(same_games[helper:(Max+helper)].iloc[i]['HT'],'Vs',same_games[helper:(Max+helper)].iloc[i]['AT'],'  ODD: ',same_games[helper:(Max+helper)].iloc[i]['ODDH_Aver.'])
                        if all(list(same_games[helper:(Max+helper)]['Won/NotWon'])):
                            print('Final Odd: ',round(np.prod(list(same_games[helper:(Max+helper)]['ODDH_Aver.'])),2),' Won!!!!!!!!!!!! :D')
                        else:
                            print('Final Odd: ',round(np.prod(list(same_games[helper:(Max+helper)]['ODDH_Aver.'])),2),' Lost :(')
                        print('-----------------------------------')
                    if all(list(same_games[helper:(Max+helper)]['Won/NotWon'])):
                        money = money + (bet_money*oddz)
                    helper += Max
                    size_aux = size_aux - Max
                oddz = round(np.prod(list(same_games[helper:]['ODDH_Aver.'])),2)
                bet_money = round(money*perc,2)
                money = money - bet_money
                if Prints:
                    print('-----------------------------------')
                    for i in range(len(same_games[helper:])):
                        print(same_games[helper:].iloc[i]['HT'],'Vs',same_games[helper:].iloc[i]['AT'],'  ODD: ',same_games[helper:].iloc[i]['ODDH_Aver.'])
                    if all(list(same_games[helper:]['Won/NotWon'])):
                        print('Final Odd: ',round(np.prod(list(same_games[helper:]['ODDH_Aver.'])),2),' Won!!!!!!!!!!!! :D')
                    else:
                        print('Final Odd: ',round(np.prod(list(same_games[helper:]['ODDH_Aver.'])),2),' Lost :(')
                    print('-----------------------------------')
                if all(list(same_games[helper:]['Won/NotWon'])):
                    money = money + (bet_money*oddz)
                d+=size
    return round(money/100,2)

In [489]:
df_noLL = pd.concat([df_L,df_Bund,df_PL,df_SA,df_Scott,df_Ered],sort=False).sort_values(['Date','Time'],ascending=[True,True]).reset_index(drop=True)

In [492]:
bet_multiple(df_noLL,Max=1,Days=1,Prints=False,Multiple_Only=False,Perc=0.1)

62.66

In [493]:
go_get(repos_Scott_P2_S)

<class 'big_dix.big_dix'>
['Tree_Forest', 'Tree_Score', 'OverAll']
Choose One of The Above: Tree_Forest
['TL50', 'TL51', 'TL52', 'TL53', 'TL54', 'TL55', 'TL56', 'TL57', 'TL58', 'TL59', 'TL60', 'TL61', 'TL62', 'TL63', 'TL64', 'TL65', 'TL66', 'TL67', 'TL68', 'TL69', 'TL70', 'TL71', 'TL72', 'TL73', 'TL74', 'TL75', 'TL76', 'TL77', 'TL78', 'TL79', 'TL80', 'TL81', 'TL82', 'TL83', 'TL84', 'TL85', 'TL86', 'TL87', 'TL88', 'TL89', 'TL90', 'TL91', 'TL92', 'TL93', 'TL94', 'TL95', 'TL96', 'TL97', 'TL98', 'TL99', 'TL100']
Choose One of The Above: TL50
['FL1', 'FL2', 'FL3', 'FL4', 'FL5', 'FL6', 'FL7', 'FL8', 'FL9', 'FL10', 'FL11', 'FL12', 'FL13', 'FL14', 'FL15', 'FL16', 'FL17', 'FL18', 'FL19', 'FL20', 'FL21', 'FL22', 'FL23', 'FL24', 'FL25', 'FL26', 'FL27', 'FL28', 'FL29', 'FL30', 'FL31', 'FL32', 'FL33', 'FL34', 'FL35', 'FL36', 'FL37', 'FL38', 'FL39', 'FL40', 'FL41', 'FL42', 'FL43', 'FL44', 'FL45', 'FL46', 'FL47', 'FL48', 'FL49', 'FL50', 'FL51', 'FL52', 'FL53', 'FL54', 'FL55', 'FL56', 'FL57', 'FL58', 

,Index,League,Date,Time,HT,AT,Tree Score %,Tree_var%50,ForestPredix_50%,ForestReal_50%,Won/NotWon,ODDH_Aver.,ODDD_Aver.,ODDA_Aver.
8,8,Scottish Premiership,2018-12-16,1500,Rangers,Hamilton Academical,54.78,53.12,1,YES,1,1.17,7.26,15.25
9,9,Scottish Premiership,2018-12-18,1945,Aberdeen,Dundee,60.21,63.62,1,YES,1,1.53,3.98,6.65
10,10,Scottish Premiership,2018-12-19,1945,Celtic,Motherwell,55.17,52.23,1,YES,1,1.19,6.57,15.53
12,12,Scottish Premiership,2018-12-22,1500,Aberdeen,Hearts,59.65,69.87,1,YES,1,1.95,3.44,3.99
13,13,Scottish Premiership,2018-12-22,1500,Celtic,Dundee,59.61,66.74,1,YES,1,1.10,9.61,22.67
27,27,Scottish Premiership,2018-12-29,1500,Kilmarnock,St Mirren,56.64,61.38,1,YES,1,1.47,4.12,7.45
29,29,Scottish Premiership,2018-12-29,1730,Hibernian,Hearts,57.78,61.61,1,NO,0,2.25,3.26,3.35
30,30,Scottish Premiership,2019-01-23,1945,Celtic,St Mirren,59.92,59.15,1,YES,1,1.10,9.32,26.86
35,35,Scottish Premiership,2019-01-23,1945,St. Johnstone,Livingston,64.28,69.64,1,YES,1,2.17,2.94,3.96
37,37,Scottish Premiership,2019-01-26,1500,Celtic,Hamilton Academical,58.29,60.04,1,YES,1,1.08,10.69,30.68


In [317]:
(41**3)*150

10338150

In [238]:
np.array(list(map(lambda x:x.days,list(abs(df_SA.iloc[0]['Date']-df_SA['Date'])))))

array([  0,   0,   1,  11,  11,  25,  32,  32,  38,  46,  46,  67,  68,
        68,  80,  81,  82,  88,  94,  95,  95,  95, 109, 110, 122, 131])

In [231]:
df_SA.loc[np.array(list(map(lambda x:x.days,list(df_SA.iloc[0]['Date']-df_SA['Date']))))<1]

,Index,League,Date,Time,HT,AT,Tree Score %,Tree_var%56,ForestPredix_55%,ForestReal_55%,Won/NotWon,ODDH_Aver.,ODDD_Aver.,ODDA_Aver.
23,23,Italian Serie A,2019-10-30,2000,Lazio,Torino,58.18,57.08,1,YES,1,1.56,4.16,5.94
24,24,Italian Serie A,2019-10-30,2000,Juventus,Genoa,67.43,66.30,1,YES,1,1.27,5.69,11.91
25,25,Italian Serie A,2019-10-31,2000,AC Milan,Spal,60.17,58.62,1,YES,1,1.47,4.25,7.29
41,41,Italian Serie A,2019-11-10,1400,Lazio,Lecce,61.47,63.05,1,YES,1,1.24,6.45,11.03
45,45,Italian Serie A,2019-11-10,1945,Juventus,AC Milan,61.66,57.00,1,YES,1,1.40,4.67,8.01
51,51,Italian Serie A,2019-11-24,1400,Roma,Brescia,61.66,65.87,1,YES,1,1.34,5.21,8.88
60,60,Italian Serie A,2019-12-01,1400,Inter Milan,Spal,68.72,72.44,1,YES,1,1.26,5.77,11.86
61,61,Italian Serie A,2019-12-01,1400,Lazio,Udinese,69.01,72.10,1,YES,1,1.30,5.42,10.00
67,67,Italian Serie A,2019-12-07,1400,Atalanta,Verona,56.74,55.89,1,YES,1,1.33,5.19,9.39
80,80,Italian Serie A,2019-12-15,1400,Juventus,Udinese,61.00,55.55,1,YES,1,1.22,6.25,14.04
